# Analysis Pipeline for Movies Dataset

This notebook preprocesses data for exploratory analysis and relationship exploration.

## Pipeline Steps:
1. Data Cleaning and Filtering
2. Handle Zero Values (Convert to NaN)
3. Extract Date Features
4. Feature Engineering - Business Metrics
5. Handle Missing Values for Analysis
6. Expand Multi-valued Categorical Features
7. Create Additional Temporal Features
8. Create Additional Derived Features


In [4]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer


ModuleNotFoundError: No module named 'sklearn'

## STEP 1: Data Cleaning and Filtering

Remove noise from the dataset:
- Drop duplicates
- Filter runtime outliers (0 < runtime <= 300)
- Filter vote_count for reliability (>= 10)
- Drop rows with invalid release dates


In [5]:
def filter_noise(df):
    """
    Remove noise from the dataset:
    - Drop duplicates
    - Filter runtime outliers (0 < runtime <= 300)
    - Filter vote_count for reliability (>= 10)
    - Drop rows with invalid release dates
    """
    df = df.copy()
    
    # Drop duplicates
    df = df.drop_duplicates()
    
    # Filter runtime outliers (0 < runtime <= 300)
    df = df[(df['runtime'] > 0) & (df['runtime'] <= 300)]
    
    # Filter vote_count for reliability (at least 10 votes)
    df = df[df['vote_count'] >= 10]
    
    # Drop rows with no release date (important for year analysis)
    temp_dates = pd.to_datetime(df['release_date'], dayfirst=True, format='mixed', errors='coerce')
    df = df[temp_dates.notna()]
    
    return df

# Load and filter data
raw_data = pd.read_csv('movies.csv')
print(f"Initial shape: {raw_data.shape}")

df_filtered = filter_noise(raw_data)
print(f"After filtering: {df_filtered.shape}")


Initial shape: (63000, 15)
After filtering: (62216, 15)


## STEP 2: Handle Zero Values (Convert to NaN)

Convert zero values to NaN for budget, revenue, and runtime.
These zeros typically represent missing data rather than true zeros.


In [6]:
def replace_zeros(df):
    """
    Convert zero values to NaN for budget, revenue, and runtime.
    These zeros typically represent missing data rather than true zeros.
    """
    df = df.copy()
    cols = ['budget', 'revenue', 'runtime']
    for col in cols:
        if col in df.columns:
            df[col] = df[col].replace(0, np.nan)
    return df

df_step2 = replace_zeros(df_filtered)
print(f"Shape: {df_step2.shape}")
print(f"Missing values after zero replacement:")
print(df_step2[['budget', 'revenue', 'runtime']].isnull().sum())


Shape: (62216, 15)
Missing values after zero replacement:
budget     48721
revenue    49348
runtime        0
dtype: int64


## STEP 3: Extract Date Features

Extract temporal features from release_date:
- Convert to datetime
- Extract release_year
- Extract release_month (month name)


In [7]:
def extract_date_features(df):
    """
    Extract temporal features from release_date:
    - Convert to datetime
    - Extract release_year
    - Extract release_month (month name)
    """
    df = df.copy()
    
    # Convert release_date to datetime
    df['release_date'] = pd.to_datetime(df['release_date'], dayfirst=True, format='mixed', errors='coerce')
    
    # Extract release year
    df['release_year'] = df['release_date'].dt.year
    
    # Extract release month (month name)
    df['release_month'] = df['release_date'].dt.month_name()
    
    return df

df_step3 = extract_date_features(df_step2)
print(f"Shape: {df_step3.shape}")
print(f"\nDate features created:")
print(df_step3[['release_date', 'release_year', 'release_month']].head())


Shape: (62216, 17)

Date features created:
  release_date  release_year release_month
0   2010-07-15          2010          July
1   2014-11-05          2014      November
2   2008-07-16          2008          July
3   2009-12-15          2009      December
4   2012-04-25          2012         April


## STEP 4: Feature Engineering - Business Metrics

Create business-related features:
- profit = revenue - budget
- roi = (revenue - budget) / budget
- primary_genre (first genre from genres list)


In [8]:
def create_business_features(df):
    """
    Create business-related features:
    - profit = revenue - budget
    - roi = (revenue - budget) / budget
    - primary_genre (first genre from genres list)
    """
    df = df.copy()
    
    # Profit and ROI
    if 'revenue' in df.columns and 'budget' in df.columns:
        df['profit'] = df['revenue'] - df['budget']
        # ROI calculation (handle division by zero)
        df['roi'] = np.where(df['budget'] != 0, 
                            (df['revenue'] - df['budget']) / df['budget'], 
                            np.nan)
    
    # Extract primary genre (first genre in the list)
    if 'genres' in df.columns:
        df['primary_genre'] = df['genres'].astype(str).str.split(',').str[0].str.strip()
        # Handle 'nan' strings
        df['primary_genre'] = df['primary_genre'].replace('nan', np.nan)
    
    return df

df_step4 = create_business_features(df_step3)
print(f"Shape: {df_step4.shape}")
print(f"\nBusiness features created:")
print(df_step4[['profit', 'roi', 'primary_genre']].head())


Shape: (62216, 20)

Business features created:
         profit        roi    primary_genre
0  6.655328e+08   4.159580           Action
1  5.367292e+08   3.252904        Adventure
2  8.195584e+08   4.430046            Drama
3  2.686706e+09  11.336312           Action
4  1.298816e+09   5.903707  Science Fiction


## STEP 5: Handle Missing Values for Analysis

Handle missing values for analysis:
- production_companies: fill with 'Unknown' (preserves missingness info)
- production_countries, genres, spoken_languages: handle NaN values


In [9]:
def handle_missing_values(df):
    """
    Handle missing values for analysis:
    - production_companies: fill with 'Unknown' (preserves information about missingness)
    - production_countries, genres, spoken_languages: handle NaN values
    """
    df = df.copy()
    
    # Fill production_companies with 'Unknown' (preserves information about missingness)
    if 'production_companies' in df.columns:
        df['production_companies'] = df['production_companies'].fillna('Unknown')
    
    # Handle other multi-valued categorical columns
    # Convert float NaN to string 'Unknown' for consistency
    for col in ['production_countries', 'genres', 'spoken_languages']:
        if col in df.columns:
            # Fill NaN with 'Unknown'
            df[col] = df[col].fillna('Unknown')
    
    return df

df_step5 = handle_missing_values(df_step4)
print(f"Shape: {df_step5.shape}")
print(f"\nMissing values after handling:")
print(df_step5[['production_companies', 'production_countries', 'genres', 'spoken_languages']].isnull().sum())


Shape: (62216, 20)

Missing values after handling:
production_companies    0
production_countries    0
genres                  0
spoken_languages        0
dtype: int64


## STEP 6: Expand Multi-valued Categorical Features

Expand multi-valued categorical features for analysis:
- Extract all genres (not just primary)
- Extract primary production country
- Extract primary spoken language
- Create counts for multi-valued features


In [10]:
def expand_categorical_features(df):
    """
    Expand multi-valued categorical features for analysis:
    - Extract all genres (not just primary)
    - Extract primary production country
    - Extract primary spoken language
    - Create counts for multi-valued features
    """
    df = df.copy()
    
    # Extract all genres and create genre count
    if 'genres' in df.columns:
        df['genre_count'] = df['genres'].astype(str).str.split(',').str.len()
        df['genre_count'] = df['genre_count'].replace(0, np.nan)  # Handle empty strings
    
    # Extract primary production country
    if 'production_countries' in df.columns:
        df['primary_country'] = df['production_countries'].astype(str).str.split(',').str[0].str.strip()
        df['primary_country'] = df['primary_country'].replace('nan', np.nan)
        df['country_count'] = df['production_countries'].astype(str).str.split(',').str.len()
        df['country_count'] = df['country_count'].replace(0, np.nan)
    
    # Extract primary spoken language
    if 'spoken_languages' in df.columns:
        df['primary_language'] = df['spoken_languages'].astype(str).str.split(',').str[0].str.strip()
        df['primary_language'] = df['primary_language'].replace('nan', np.nan)
        df['language_count'] = df['spoken_languages'].astype(str).str.split(',').str.len()
        df['language_count'] = df['language_count'].replace(0, np.nan)
    
    # Extract primary production company
    if 'production_companies' in df.columns:
        df['primary_company'] = df['production_companies'].astype(str).str.split(',').str[0].str.strip()
        df['primary_company'] = df['primary_company'].replace('nan', 'Unknown')
        df['company_count'] = df['production_companies'].astype(str).str.split(',').str.len()
        df['company_count'] = df['company_count'].replace(0, np.nan)
    
    return df

df_step6 = expand_categorical_features(df_step5)
print(f"Shape: {df_step6.shape}")
print(f"\nNew categorical features:")
print(df_step6[['genre_count', 'primary_country', 'primary_language', 'primary_company']].head())


Shape: (62216, 27)

New categorical features:
   genre_count           primary_country primary_language     primary_company
0            3            United Kingdom          English  Legendary Pictures
1            3            United Kingdom          English  Legendary Pictures
2            4            United Kingdom          English           DC Comics
3            4  United States of America          English  Dune Entertainment
4            3  United States of America          English      Marvel Studios


## STEP 7: Create Additional Temporal Features

Create additional temporal features:
- release_quarter (Q1-Q4)
- release_decade
- is_summer_blockbuster (May-August)
- is_holiday_season (November-December)


In [11]:
def create_temporal_features(df):
    """
    Create additional temporal features:
    - release_quarter (Q1-Q4)
    - release_decade
    - is_summer_blockbuster (May-August)
    - is_holiday_season (November-December)
    """
    df = df.copy()
    
    if 'release_date' in df.columns:
        # Release quarter
        df['release_quarter'] = df['release_date'].dt.quarter
        df['release_quarter'] = df['release_quarter'].map({1: 'Q1', 2: 'Q2', 3: 'Q3', 4: 'Q4'})
        
        # Release decade
        df['release_decade'] = (df['release_year'] // 10) * 10
        
        # Summer blockbuster flag (May-August)
        df['is_summer_blockbuster'] = df['release_date'].dt.month.isin([5, 6, 7, 8])
        
        # Holiday season flag (November-December)
        df['is_holiday_season'] = df['release_date'].dt.month.isin([11, 12])
    
    return df

df_step7 = create_temporal_features(df_step6)
print(f"Shape: {df_step7.shape}")
print(f"\nNew temporal features:")
print(df_step7[['release_quarter', 'release_decade', 'is_summer_blockbuster', 'is_holiday_season']].head())


Shape: (62216, 31)

New temporal features:
  release_quarter  release_decade  is_summer_blockbuster  is_holiday_season
0              Q3            2010                   True              False
1              Q4            2010                  False               True
2              Q3            2000                   True              False
3              Q4            2000                  False               True
4              Q2            2010                  False              False


## STEP 8: Create Additional Derived Features

Create additional derived features for analysis:
- budget_revenue_ratio
- vote_reliability (weighted vote average)
- runtime_bins (Short/Medium/Long)
- financial_success flag


In [12]:
def create_derived_features(df):
    """
    Create additional derived features for analysis:
    - budget_revenue_ratio
    - vote_reliability (weighted vote average)
    - runtime_bins (Short/Medium/Long)
    - financial_success flag
    """
    df = df.copy()
    
    # Budget to revenue ratio
    if 'budget' in df.columns and 'revenue' in df.columns:
        df['budget_revenue_ratio'] = np.where(df['budget'] > 0,
                                             df['revenue'] / df['budget'],
                                             np.nan)
    
    # Vote reliability (weighted by vote count)
    if 'vote_average' in df.columns and 'vote_count' in df.columns:
        df['vote_reliability'] = df['vote_average'] * np.log1p(df['vote_count'])
    
    # Runtime bins
    if 'runtime' in df.columns:
        df['runtime_bins'] = pd.cut(df['runtime'], 
                                   bins=[0, 90, 120, 300],
                                   labels=['Short', 'Medium', 'Long'],
                                   include_lowest=True)
    
    # Financial success flag (revenue > 2 * budget)
    if 'revenue' in df.columns and 'budget' in df.columns:
        df['financial_success'] = df['revenue'] > (2 * df['budget'])
    
    return df

df_step8 = create_derived_features(df_step7)
print(f"Shape: {df_step8.shape}")
print(f"\nNew derived features:")
print(df_step8[['budget_revenue_ratio', 'vote_reliability', 'runtime_bins', 'financial_success']].head())


Shape: (62216, 35)

New derived features:
   budget_revenue_ratio  vote_reliability runtime_bins  financial_success
0              5.159580         87.392079         Long               True
1              4.252904         87.462800         Long               True
2              5.430046         87.923927         Long               True
3             12.336312         78.023108         Long               True
4              6.903707         79.264916         Long               True


## Final Step: Save Preprocessed Dataset

Save the cleaned and preprocessed dataset for analysis.


In [13]:
# Final dataset for analysis
df_analysis = df_step8.copy()

print("Preprocessing Complete!")
print(f"Final shape: {df_analysis.shape}")
print(f"\nColumns: {df_analysis.columns.tolist()}")

# Save the new cleaned and preprocessed dataset
df_analysis.to_csv('movies_cleaned_analysis.csv', index=False)
print("\nThe cleaned dataset is saved as 'movies_cleaned_analysis.csv'")

# Display first few rows
df_analysis.head()


Preprocessing Complete!
Final shape: (62216, 35)

Columns: ['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date', 'revenue', 'runtime', 'budget', 'original_language', 'popularity', 'genres', 'production_companies', 'production_countries', 'spoken_languages', 'release_year', 'release_month', 'profit', 'roi', 'primary_genre', 'genre_count', 'primary_country', 'country_count', 'primary_language', 'language_count', 'primary_company', 'company_count', 'release_quarter', 'release_decade', 'is_summer_blockbuster', 'is_holiday_season', 'budget_revenue_ratio', 'vote_reliability', 'runtime_bins', 'financial_success']

The cleaned dataset is saved as 'movies_cleaned_analysis.csv'


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,original_language,...,primary_company,company_count,release_quarter,release_decade,is_summer_blockbuster,is_holiday_season,budget_revenue_ratio,vote_reliability,runtime_bins,financial_success
0,27205,Inception,8.364,34495,Released,2010-07-15,8.255328e+08,148,160000000.0,en,...,Legendary Pictures,3,Q3,2010,True,False,5.159580,87.392079,Long,True
1,157336,Interstellar,8.417,32571,Released,2014-11-05,7.017292e+08,169,165000000.0,en,...,Legendary Pictures,3,Q4,2010,False,True,4.252904,87.462800,Long,True
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1.004558e+09,152,185000000.0,en,...,DC Comics,5,Q3,2000,True,False,5.430046,87.923927,Long,True
3,19995,Avatar,7.573,29815,Released,2009-12-15,2.923706e+09,162,237000000.0,en,...,Dune Entertainment,4,Q4,2000,False,True,12.336312,78.023108,Long,True
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1.518816e+09,143,220000000.0,en,...,Marvel Studios,1,Q2,2010,False,False,6.903707,79.264916,Long,True
